In [2]:
!pip install bitsandbytes datasets sentence-transformers evaluate bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requir

In [1]:
from google.colab import userdata


from huggingface_hub import login
login(token=userdata.get('huggingface_login'))

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, util
import evaluate

bertscore_metric = evaluate.load("bertscore")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
wikipedia_tc = load_dataset('rag-datasets/rag-mini-wikipedia', 'text-corpus')
wikipedia_qa = load_dataset('rag-datasets/rag-mini-wikipedia', 'question-answer')

README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/3200 [00:00<?, ? examples/s]

part.0.parquet:   0%|          | 0.00/54.4k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/918 [00:00<?, ? examples/s]

In [5]:
wikipedia_qa['test']['question'][:5]

['Was Abraham Lincoln the sixteenth President of the United States?',
 'Did Lincoln sign the National Banking Act of 1863?',
 'Did his mother die of pneumonia?',
 "How many long was Lincoln's formal education?",
 'When did Lincoln begin his political career?']

In [6]:
wikipedia_qa['test']['answer'][:5]

['yes', 'yes', 'no', '18 months', '1832']

In [7]:
documents = wikipedia_tc['passages']['passage']

In [8]:
documents[:5]

['Uruguay (official full name in  ; pron.  , Eastern Republic of  Uruguay) is a country located in the southeastern part of South America.  It is home to 3.3 million people, of which 1.7 million live in the capital Montevideo and its metropolitan area.',
 'It is bordered by Brazil to the north, by Argentina across the bank of both the Uruguay River to the west and the estuary of RÃ\xado de la Plata to the southwest, and the South Atlantic Ocean to the southeast. It is the second smallest independent country in South America, larger only than Suriname and the French overseas department of French Guiana.',
 'Montevideo was founded by the Spanish in the early 18th century as a military stronghold. Uruguay won its independence in 1828 following a three-way struggle between Spain, Argentina and Brazil. It is a constitutional democracy, where the president fulfills the roles of both head of state and head of government',
 'The economy is largely based in agriculture (making up 10% of the GDP

In [ ]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
model_name = 'llama-2-7b-chat-hf'
tokenizer_name = 'llama-2-7b-chat-hf'
padding_size = 1024

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          model_max_length=padding_size)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

# Задача 1

In [ ]:
import evaluate

bertscore_metric = evaluate.load("bertscore")

all_bertscores = []

for i, question in enumerate(wikipedia_qa['test']['question'][:50]):
    emb = embeddings_model.encode(question, batch_size=64, show_progress_bar=True)
    context = util.semantic_search(emb, document_embeddings, top_k=5)
    ctx_doc_ids = [c['corpus_id'] for c in context[0]]
    ctx_docs = [documents[j] for j in ctx_doc_ids]

    context_text = "\n".join(
        [f"Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
    )

    prompt = (
        f"Answer the following question using the provided context. "
        f"Provide the answer using this template: your answer here.\n\n"
        f"Context:\n{context_text}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']

    input_ids = input_ids.to(model.device)
    attention_masks = attention_masks.to(model.device)

    output_ids = model.generate(input_ids=input_ids, attention_mask=attention_masks)
    generated_answer = tokenizer.decode(
        output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
    ).strip()

    reference_answers = wikipedia_qa['test']['answer'][i]
    references = [reference_answers]

    bertscore_result = bertscore_metric.compute(
        predictions=[generated_answer],
        references=references,
        lang="en"
    )

    all_bertscores.append(bertscore_result["f1"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
average_bertscore = sum(all_bertscores) / len(all_bertscores)
print(f"Average BERTScore: {average_bertscore:.4f}")

Average BERTScore: 0.7782


# Не можев да евалуирам со BLEU, добивав грешка која никако не успеав да ја решам, па затоа користам само BERTScore низ целата лаб вежба. Исто така премногу долго се извршува тренирањето, па затоа лимитирав на само 50 примероци

# Тест со DistilRoBERTa

In [ ]:
embeddings_model = SentenceTransformer('all-distilroberta-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
model_name = 'llama-2-7b-chat-hf'
tokenizer_name = 'llama-2-7b-chat-hf'
padding_size = 1024

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          model_max_length=padding_size)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import evaluate

bertscore_metric = evaluate.load("bertscore")

all_bertscores = []

for i, question in enumerate(wikipedia_qa['test']['question'][:50]):
    emb = embeddings_model.encode(question, batch_size=64, show_progress_bar=True)
    context = util.semantic_search(emb, document_embeddings, top_k=5)
    ctx_doc_ids = [c['corpus_id'] for c in context[0]]
    ctx_docs = [documents[j] for j in ctx_doc_ids]

    context_text = "\n".join(
        [f"Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
    )

    prompt = (
        f"Answer the following question using the provided context. "
        f"Provide the answer using this template: your answer here.\n\n"
        f"Context:\n{context_text}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']

    input_ids = input_ids.to(model.device)
    attention_masks = attention_masks.to(model.device)

    output_ids = model.generate(input_ids=input_ids, attention_mask=attention_masks)
    generated_answer = tokenizer.decode(
        output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
    ).strip()

    reference_answers = wikipedia_qa['test']['answer'][i]
    references = [reference_answers]

    bertscore_result = bertscore_metric.compute(
        predictions=[generated_answer],
        references=references,
        lang="en"
    )

    all_bertscores.append(bertscore_result["f1"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
average_bertscore = sum(all_bertscores) / len(all_bertscores)
print(f"Average BERTScore with DistilRoBERTa: {average_bertscore:.4f}")

Average BERTScore with DistilRoBERTa: 0.7787


# Zero-Shot тест:

In [ ]:
model_name = 'llama-2-7b-chat-hf'
tokenizer_name = 'llama-2-7b-chat-hf'
padding_size = 1024

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          model_max_length=padding_size)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [ ]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import evaluate

bertscore_metric = evaluate.load("bertscore")

all_bertscores = []

qa_pairs = []

for i, question in enumerate(wikipedia_qa['test']['question'][:50]):
    emb = embeddings_model.encode(question, batch_size=64, show_progress_bar=True)

    prompt = (
        f"Answer the following question. "
        f"Provide the answer using this template: your answer here.\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids']
    attention_masks = inputs['attention_mask']

    input_ids = input_ids.to(model.device)
    attention_masks = attention_masks.to(model.device)

    output_ids = model.generate(input_ids=input_ids, attention_mask=attention_masks)
    generated_answer = tokenizer.decode(
        output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
    ).strip()

    reference_answers = wikipedia_qa['test']['answer'][i]
    references = [reference_answers]

    bertscore_result = bertscore_metric.compute(
        predictions=[generated_answer],
        references=references,
        lang="en"
    )

    all_bertscores.append(bertscore_result["f1"][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
average_bertscore = sum(all_bertscores) / len(all_bertscores)
print(f"Average Zero-Shot BERTScore: {average_bertscore:.4f}")

Average Zero-Shot BERTScore: 0.7971


# Кај овој модел изгледа дека нема значителна разлика во резултатие, барем не со лимит од првите 50 прашања. Највисоки резулати добива со Zero-Shot prompting, најниски со all-MiniLM-L6-v2, но сепак трите опции даваат сличен резултат.

# Задача 2

In [ ]:
model_name = 'Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          model_max_length=1024)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [ ]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
document_embeddings = embeddings_model.encode(documents, batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
bertscore_metric = evaluate.load("bertscore")

In [ ]:
num_docs = 5
all_bertscores = []

for i, question in enumerate(wikipedia_qa['test']['question'][:50]):
    question_embedding = embeddings_model.encode(question, show_progress_bar=False)

    context = util.semantic_search(question_embedding, document_embeddings, top_k=num_docs)
    ctx_doc_ids = [c['corpus_id'] for c in context[0]]
    ctx_docs = [documents[j] for j in ctx_doc_ids]

    context_text = "\n".join(
        [f"Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
    )

    prompt = (
        f"Answer the following question using the provided context. "
        f"Provide the answer using this template: your answer here.\n\n"
        f"Context:\n{context_text}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(model.device)

    output_ids = model.generate(input_ids=input_ids)
    generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    reference_answers = wikipedia_qa['test']['answer'][i]
    bertscore_result = bertscore_metric.compute(
        predictions=[generated_answer],
        references=[reference_answers],
        lang="en"
    )

    all_bertscores.append(bertscore_result["f1"][0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to

In [ ]:
average_bertscore = sum(all_bertscores) / len(all_bertscores)
print(f"Average BERTScore (5 Docs): {average_bertscore:.4f}")

Average BERTScore (5 Docs): 0.7789


In [ ]:
bertscore_results = {}

for num_docs in [3, 7, 10]:
    all_bertscores = []

    for i, question in enumerate(wikipedia_qa['test']['question'][:50]):
        question_embedding = embeddings_model.encode(question, show_progress_bar=False)

        context = util.semantic_search(question_embedding, document_embeddings, top_k=num_docs)
        ctx_doc_ids = [c['corpus_id'] for c in context[0]]
        ctx_docs = [documents[j] for j in ctx_doc_ids]

        context_text = "\n".join(
            [f"Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
        )

        prompt = (
            f"Answer the following question using the provided context. "
            f"Provide the answer using this template: your answer here.\n\n"
            f"Context:\n{context_text}\n\n"
            f"Question: {question}\n\n"
            f"Answer:"
        )

        inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
        input_ids = inputs['input_ids'].to(model.device)

        output_ids = model.generate(input_ids=input_ids)
        generated_answer = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        reference_answers = wikipedia_qa['test']['answer'][i]
        bertscore_result = bertscore_metric.compute(
            predictions=[generated_answer],
            references=[reference_answers],
            lang="en"
        )

        all_bertscores.append(bertscore_result["f1"][0])

    average_bertscore = sum(all_bertscores) / len(all_bertscores)
    bertscore_results[num_docs] = {
        "average": average_bertscore,
        "individual": all_bertscores
    }

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene


#### Results for num_docs = 3 ####
Average BERTScore: 0.7823
Individual BERTScores: [0.7699517607688904, 0.7830671072006226, 0.7819521427154541, 0.7995295524597168, 0.7951396107673645, 0.8350945711135864, 0.84666907787323, 0.7783479690551758, 0.7764112949371338, 0.7715566754341125, 0.7828130722045898, 0.7875431180000305, 0.7617126703262329, 0.789954423904419, 0.7985399961471558, 0.7811387181282043, 0.7685939073562622, 0.8010284304618835, 0.7845140695571899, 0.7904607057571411, 0.7847912907600403, 0.7802873849868774, 0.7858946323394775, 0.7951681017875671, 0.7866140007972717, 0.7854099273681641, 0.7757708430290222, 0.7782082557678223, 0.7324160933494568, 0.775176465511322, 0.7670627236366272, 0.7797197699546814, 0.7676166892051697, 0.7653821706771851, 0.7640089392662048, 0.7715152502059937, 0.7662839889526367, 0.7738500833511353, 0.7594689130783081, 0.7657785415649414, 0.770124614238739, 0.8500241637229919, 0.7507221698760986, 0.7806894779205322, 0.7880855798721313, 0.7745860815048218,

In [ ]:
for num_docs, results in bertscore_results.items():
    print(f"\nResults for num_docs = {num_docs}")
    print(f"Average BERTScore: {results['average']:.4f}")
    print(f"Individual BERTScores: {results['individual']}")



Results for num_docs = 3
Average BERTScore: 0.7823
Individual BERTScores: [0.7699517607688904, 0.7830671072006226, 0.7819521427154541, 0.7995295524597168, 0.7951396107673645, 0.8350945711135864, 0.84666907787323, 0.7783479690551758, 0.7764112949371338, 0.7715566754341125, 0.7828130722045898, 0.7875431180000305, 0.7617126703262329, 0.789954423904419, 0.7985399961471558, 0.7811387181282043, 0.7685939073562622, 0.8010284304618835, 0.7845140695571899, 0.7904607057571411, 0.7847912907600403, 0.7802873849868774, 0.7858946323394775, 0.7951681017875671, 0.7866140007972717, 0.7854099273681641, 0.7757708430290222, 0.7782082557678223, 0.7324160933494568, 0.775176465511322, 0.7670627236366272, 0.7797197699546814, 0.7676166892051697, 0.7653821706771851, 0.7640089392662048, 0.7715152502059937, 0.7662839889526367, 0.7738500833511353, 0.7594689130783081, 0.7657785415649414, 0.770124614238739, 0.8500241637229919, 0.7507221698760986, 0.7806894779205322, 0.7880855798721313, 0.7745860815048218, 0.7858604

# Нема значителна разлика со повеќе или помалку документи.

In [12]:
!7z x 'text detoxification.zip'


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 507343 bytes (496 KiB)

Extracting archive: text detoxification.zip
--
Path = text detoxification.zip
Type = zip
Physical Size = 507343

  0%    Everything is Ok

Files: 3
Size:       1630442
Compressed: 507343


In [17]:
files = ["train_en.txt", "test_en.txt", "val_en.txt"]

new_header = "toxic\tneutral\n"

for file in files:
    with open(file, "r") as f:
        lines = f.readlines()

    lines[0] = new_header

    with open(file, "w") as f:
        f.writelines(lines)

In [13]:
model_name = 'llama-2-7b-chat-hf'
padding_size = 1024
tokenizer = AutoTokenizer.from_pretrained(f'meta-llama/{model_name}',
                                          use_auth_token=True,
                                          model_max_length=1024)
tokenizer.pad_token = tokenizer.bos_token
tokenizer.padding_side = 'left'

In [14]:
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4')

In [15]:
model = AutoModelForCausalLM.from_pretrained(f'meta-llama/{model_name}',
                                             device_map='cuda:0',
                                             quantization_config=quantization_config)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [16]:
embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
import pandas as pd

train_data = pd.read_csv('train_en.txt', sep='\t', names=['toxic', 'neutral'])
test_data = pd.read_csv('test_en.txt', sep='\t', names=['toxic', 'neutral'])
val_data = pd.read_csv('val_en.txt', sep='\t', names=['toxic', 'neutral'])

print(train_data.head())


                                               toxic  \
0                                              toxic   
1                           he had steel balls too !   
2  dude should have been taken to api , he would ...   
3  im not gonna sell the fucking picture , i just...   
4  the garbage that is being created by cnn and o...   

                                             neutral  
0                                            neutral  
1                                  he was brave too!  
2  It would have been good if he went to api. He ...  
3  I'm not gonna sell the picture, i just want to...  
4  the news that is being created by cnn and othe...  


In [20]:
from sentence_transformers import util

neutral_documents = train_data['neutral'].tolist()
document_embeddings = embeddings_model.encode(neutral_documents, batch_size=64, show_progress_bar=True)

toxic_sentences = test_data['toxic'].tolist()
neutral_references = test_data['neutral'].tolist()

detoxified_sentences = []

for i, toxic_text in enumerate(toxic_sentences[:50]):
    toxic_embedding = embeddings_model.encode(toxic_text, show_progress_bar=False)
    context = util.semantic_search(toxic_embedding, document_embeddings, top_k=5)
    ctx_doc_ids = [c['corpus_id'] for c in context[0]]
    ctx_docs = [neutral_documents[j] for j in ctx_doc_ids]

    context_text = "\n".join(
        [f"Neutral Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
    )

    prompt = (
        f"Convert the following toxic text to neutral using the provided context. "
        f"Provide the answer using this template: your neutral text here.\n\n"
        f"Context:\n{context_text}\n\n"
        f"Toxic Text: {toxic_text}\n\n"
        f"Neutral Text:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(model.device)

    output_ids = model.generate(input_ids=input_ids)
    detoxified_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    detoxified_sentences.append(detoxified_text)

for idx in range(5):
    print(f"Toxic: {toxic_sentences[idx]}")
    print(f"Detoxified: {detoxified_sentences[idx]}")
    print(f"Reference Neutral: {neutral_references[idx]}")
    print()


Batches:   0%|          | 0/196 [00:00<?, ?it/s]

Toxic: toxic
Detoxified: Convert the following toxic text to neutral using the provided context. Provide the answer using this template: your neutral text here.

Context:
Neutral Document 1: In a thread like this , being that vague is toxic.
Neutral Document 2: to say that masculinity is toxic is wrongly manifested in a malicious way .
Neutral Document 3: In a thread like this, being that vague is pretty toxic
Neutral Document 4: Hey bho that is poison
Neutral Document 5: In a thread like this, being vague is pretty toxic

Toxic Text: toxic

Neutral Text: unclear

Your Neutral Text Here: In a thread like this, being vague is unclear.
Reference Neutral: neutral

Toxic: this is a free country that protectsamendment rights from little commies like you .
Detoxified: Convert the following toxic text to neutral using the provided context. Provide the answer using this template: your neutral text here.

Context:
Neutral Document 1: This is a free country that protects amendments rights from c

In [26]:
bertscore_result = bertscore_metric.compute(
    predictions=detoxified_sentences,
    references=neutral_references[:50],
    lang='en'
)
average_bertscore = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])

print(f'Average BERTScore for llama2 with minilm: {average_bertscore:.4f}')

Average BERTScore for llama2 with minilm: 0.8318


# Тест со DistilRoBERTa

In [27]:
embeddings_model = SentenceTransformer('all-distilroberta-v1')
document_embeddings = embeddings_model.encode(neutral_documents, batch_size=64, show_progress_bar=True)

toxic_sentences = test_data['toxic'].tolist()
neutral_references = test_data['neutral'].tolist()

detoxified_sentences = []

for i, toxic_text in enumerate(toxic_sentences[:50]):
    toxic_embedding = embeddings_model.encode(toxic_text, show_progress_bar=False)
    context = util.semantic_search(toxic_embedding, document_embeddings, top_k=5)
    ctx_doc_ids = [c['corpus_id'] for c in context[0]]
    ctx_docs = [neutral_documents[j] for j in ctx_doc_ids]

    context_text = "\n".join(
        [f"Neutral Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
    )

    prompt = (
        f"Convert the following toxic text to neutral using the provided context. "
        f"Provide the answer using this template: your neutral text here.\n\n"
        f"Context:\n{context_text}\n\n"
        f"Toxic Text: {toxic_text}\n\n"
        f"Neutral Text:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(model.device)

    output_ids = model.generate(input_ids=input_ids)
    detoxified_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    detoxified_sentences.append(detoxified_text)

for idx in range(5):
    print(f"Toxic: {toxic_sentences[idx]}")
    print(f"Detoxified: {detoxified_sentences[idx]}")
    print(f"Reference Neutral: {neutral_references[idx]}")
    print()


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/196 [00:00<?, ?it/s]

Toxic: toxic
Detoxified: Convert the following toxic text to neutral using the provided context. Provide the answer using this template: your neutral text here.

Context:
Neutral Document 1: Hey bho that is poison
Neutral Document 2: In a thread like this, being that vague is pretty toxic
Neutral Document 3: In a thread like this , being that vague is toxic.
Neutral Document 4: In a thread like this, being vague is pretty toxic
Neutral Document 5: to say that masculinity is toxic is wrongly manifested in a malicious way .

Toxic Text: toxic

Neutral Text: constructive

Your Neutral Text Here: In a thread like this, being constructive and specific with one's opinions is important for respectful and productive dialogue.
Reference Neutral: neutral

Toxic: this is a free country that protectsamendment rights from little commies like you .
Detoxified: Convert the following toxic text to neutral using the provided context. Provide the answer using this template: your neutral text here.

Cont

In [28]:
bertscore_result = bertscore_metric.compute(
    predictions=detoxified_sentences,
    references=neutral_references[:50],
    lang='en'
)
average_bertscore = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])

print(f'Average BERTScore for llama2 with DistilRoBERTa: {average_bertscore:.4f}')

Average BERTScore for llama2 with DistilRoBERTa: 0.8296


# Тест со различни големини на контестот

In [29]:
for num_docs in [3, 7, 10]:
    detoxified_sentences = []

    for i, toxic_text in enumerate(toxic_sentences[:50]):
        toxic_embedding = embeddings_model.encode(toxic_text, show_progress_bar=False)
        context = util.semantic_search(toxic_embedding, document_embeddings, top_k=num_docs)
        ctx_doc_ids = [c['corpus_id'] for c in context[0]]
        ctx_docs = [neutral_documents[j] for j in ctx_doc_ids]

        context_text = "\n".join(
            [f"Neutral Document {j + 1}: {doc}" for j, doc in enumerate(ctx_docs)]
        )

        prompt = (
            f"Convert the following toxic text to neutral using the provided context. "
            f"Provide the answer using this template: your neutral text here.\n\n"
            f"Context:\n{context_text}\n\n"
            f"Toxic Text: {toxic_text}\n\n"
            f"Neutral Text:"
        )

        inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
        input_ids = inputs['input_ids'].to(model.device)

        output_ids = model.generate(input_ids=input_ids)
        detoxified_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

        detoxified_sentences.append(detoxified_text)

    print(f"Results for num_docs = {num_docs}:")
    for idx in range(3):
        print(f"Toxic: {toxic_sentences[idx]}")
        print(f"Detoxified: {detoxified_sentences[idx]}")
        print(f"Reference Neutral: {neutral_references[idx]}")
        print()
    print(f"BERTScore for context size: {num_docs}")
    bertscore_result = bertscore_metric.compute(
    predictions=detoxified_sentences,
    references=neutral_references[:50],
    lang='en'
    )
    average_bertscore = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])

    print(f"BERTScore: {average_bertscore:.4f}")


Results for num_docs = 3:
Toxic: toxic
Detoxified: Convert the following toxic text to neutral using the provided context. Provide the answer using this template: your neutral text here.

Context:
Neutral Document 1: Hey bho that is poison
Neutral Document 2: In a thread like this, being that vague is pretty toxic
Neutral Document 3: In a thread like this , being that vague is toxic.

Toxic Text: toxic

Neutral Text: neutral
Reference Neutral: neutral

Toxic: this is a free country that protectsamendment rights from little commies like you .
Detoxified: Convert the following toxic text to neutral using the provided context. Provide the answer using this template: your neutral text here.

Context:
Neutral Document 1: This is a free country that protects amendments rights from communists like you.
Neutral Document 2: Yet its the liberals taking away freedoms.
Neutral Document 3: things like this is why i 'm so attached to the 2 nd amendment

Toxic Text: this is a free country that protec

In [30]:
detoxified_sentences = []

for toxic_text in toxic_sentences[:50]:
    prompt = (
        f"Convert the following toxic text to neutral. "
        f"Provide the answer using this template: your neutral text here.\n\n"
        f"Toxic Text: {toxic_text}\n\n"
        f"Neutral Text:"
    )

    inputs = tokenizer(prompt, truncation=True, padding=True, return_tensors='pt')
    input_ids = inputs['input_ids'].to(model.device)

    output_ids = model.generate(input_ids=input_ids)
    detoxified_text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()

    detoxified_sentences.append(detoxified_text)

for idx in range(5):
    print(f"Toxic: {toxic_sentences[idx]}")
    print(f"Detoxified (Zero-Shot): {detoxified_sentences[idx]}")
    print(f"Reference Neutral: {neutral_references[idx]}")
    print()


Toxic: toxic
Detoxified (Zero-Shot): Convert the following toxic text to neutral. Provide the answer using this template: your neutral text here.

Toxic Text: toxic

Neutral Text: neutral

Your Neutral Text Here: Your neutral text here.
Reference Neutral: neutral

Toxic: this is a free country that protectsamendment rights from little commies like you .
Detoxified (Zero-Shot): Convert the following toxic text to neutral. Provide the answer using this template: your neutral text here.

Toxic Text: this is a free country that protectsamendment rights from little commies like you .

Neutral Text: This is a country that values and respects the rights of all individuals, regardless of their beliefs or opinions.
Reference Neutral: This is a free country.

Toxic: generous seperation packages my ass !
Detoxified (Zero-Shot): Convert the following toxic text to neutral. Provide the answer using this template: your neutral text here.

Toxic Text: generous seperation packages my ass !

Neutral Te

In [31]:
bertscore_result = bertscore_metric.compute(
    predictions=detoxified_sentences,
    references=neutral_references[:50],
    lang='en'
)
average_bertscore = sum(bertscore_result["f1"]) / len(bertscore_result["f1"])

print(f'Average BERTScore for llama2 with zero-shot prompting: {average_bertscore:.4f}')

Average BERTScore for llama2 with zero-shot prompting: 0.8581


# Најдобри перформанси има моделот со zero-shot prompting, но пак, разликите се многу мали помеѓу различните конфигурации на архитектурата. Можеби доколку брзо се извршуваа кодовите, со повеќе примероци би добиле поразлилни и поточни резултати. Сепак, и од испечатените референци и детоксифицирани текстови може да се согледа дека моделот добро ги препознава токсичните елементи и ги отстранува или изменува.